In [1]:
from os.path import join
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode

In [2]:
# Dataset definition
class ImageDataset(Dataset):
    def __init__(self, im_dir):
        df = pd.read_csv(join(im_dir, "label.csv"), sep=";")
        self.images = df["FileName"].apply(lambda im: join(im_dir, im))
        self.labels = df["Angle"]
        
    def __len__(self) -> int:
        return len(self.labels)
    
    def __getitem__(self, index) -> (torch.Tensor, float):
        image = read_image(self.images[index], ImageReadMode.GRAY) / 255
        label = self.labels[index]
        return image, torch.Tensor([label])

    
batch_size = 48

train_dataloader = DataLoader(ImageDataset("../data/train/"), batch_size=batch_size, shuffle=True)
test_dataloader  = DataLoader(ImageDataset("../data/test/"), batch_size=batch_size, shuffle=True)

In [3]:
# Model definition
class Regression(nn.Module):
    def __init__(self):
        super(Regression, self).__init__()
        self.c1 = nn.Conv2d(1, 4, kernel_size=3)
        self.p1 = nn.MaxPool2d(2)
        
        self.c2 = nn.Conv2d(4, 8, kernel_size=3)
        self.p2 = nn.MaxPool2d(2)
        self.d1 = nn.Dropout2d(0.4)
        
        self.c3 = nn.Conv2d(8, 16, kernel_size=3)
        self.p3 = nn.MaxPool2d(2)
        self.d2 = nn.Dropout2d(0.4)
        
        self.f1 = nn.Flatten()
        self.l1 = nn.Linear(45 * 50 * 16, 64)
        self.d3 = nn.Dropout()
        
        self.l2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.p1(F.relu(self.c1(x)))
        x = self.d1(self.p2(F.relu(self.c2(x))))
        x = self.d2(self.p3(F.relu(self.c3(x))))
        x = self.d3(F.relu(self.l1(self.f1(x))))
        x = F.relu(self.l2(x))
        return x

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def run_epoch(model, dataloader, criterion, optimizer, verbose=True):
    if optimizer:
        model.train()
    else:
        model.eval()
    
    run_loss, run_error = 0, 0
    for x_batch, y_batch in dataloader:
        if optimizer:
            optimizer.zero_grad()
        
        y_pred = model(x_batch.to(device)).to("cpu")
        loss = criterion(y_pred, y_batch)
        
        if optimizer:
            loss.backward()
            optimizer.step()
        
        run_loss += loss.item()
        run_error += (y_pred - y_batch).abs().sum().item()
        
    if verbose:
        print("Loss: {:10.3f} | MAE: {:7.3f}".format(run_loss * batch_size / len(dataloader.dataset), run_error / len(dataloader.dataset)))
    return run_error / len(dataloader.dataset)

In [5]:
# Training part

In [6]:
model = Regression()
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

print(model, "on", device, "with", sum(p.numel() for p in model.parameters() if p.requires_grad), "parameters")

Regression(
  (c1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout2d(p=0.4, inplace=False)
  (c3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): Dropout2d(p=0.4, inplace=False)
  (f1): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=36000, out_features=64, bias=True)
  (d3): Dropout(p=0.5, inplace=False)
  (l2): Linear(in_features=64, out_features=1, bias=True)
) on cuda:0 with 2305633 parameters


In [7]:
n_epochs = 20
for n in range(n_epochs):
    print("Epoch {:4d}".format(n), end=" | ")
    run_epoch(model, train_dataloader, criterion, optimizer)
print("Test", end = " | ")
mae = run_epoch(model, test_dataloader, criterion, optimizer=None)
torch.save(model.state_dict(), "./model-{:.3f}.ckpt".format(mae))

Epoch    0 | Loss:  13173.362 | MAE:  98.675
Epoch    1 | Loss:   4466.504 | MAE:  55.074
Epoch    2 | Loss:   3676.659 | MAE:  49.304
Epoch    3 | Loss:   3289.979 | MAE:  46.353
Epoch    4 | Loss:   3129.525 | MAE:  45.128
Epoch    5 | Loss:   3352.807 | MAE:  46.581
Epoch    6 | Loss:   3214.933 | MAE:  45.602
Epoch    7 | Loss:   3064.295 | MAE:  44.867
Epoch    8 | Loss:   2910.362 | MAE:  43.169
Epoch    9 | Loss:   2920.530 | MAE:  43.533
Epoch   10 | Loss:   3043.188 | MAE:  44.557
Epoch   11 | Loss:   2842.424 | MAE:  43.098
Epoch   12 | Loss:   3010.028 | MAE:  43.931
Epoch   13 | Loss:   2841.475 | MAE:  42.835
Epoch   14 | Loss:   2873.873 | MAE:  43.321
Epoch   15 | Loss:   2824.128 | MAE:  42.984
Epoch   16 | Loss:   2893.802 | MAE:  43.459
Epoch   17 | Loss:   2935.064 | MAE:  43.559
Epoch   18 | Loss:   2866.750 | MAE:  43.232
Epoch   19 | Loss:   2875.120 | MAE:  42.824
Test | Loss:    404.912 | MAE:  17.860


In [5]:
# Inference part

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Regression()
model.to(device)

criterion = nn.MSELoss()
model.load_state_dict(torch.load("model-final-15.318.ckpt"))

print(model, "on", device, "with", sum(p.numel() for p in model.parameters() if p.requires_grad), "parameters")

Regression(
  (c1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout2d(p=0.4, inplace=False)
  (c3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): Dropout2d(p=0.4, inplace=False)
  (f1): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=36000, out_features=64, bias=True)
  (d3): Dropout(p=0.5, inplace=False)
  (l2): Linear(in_features=64, out_features=1, bias=True)
) on cuda:0 with 2305633 parameters


In [7]:
print("Model's performance on test dataset:", end=" | ")
run_epoch(model, test_dataloader, criterion, optimizer=None)

Model's performance on test dataset: | Loss:    274.606 | MAE:  15.318


15.3178412630627